In [1]:
import pandas as pd
import geopandas as gpd
import toml
from pathlib import Path
from sqlalchemy import create_engine
from functions import process_network_summary

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.1f}'.format)

In [2]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]

## Congestion Levels by County

Congestion levels are defined by the ratio of congested speed to posted speed limits:

- Minimal: Ratio > 0.70
- Moderate: Ratio > 0.50 and <=0.70
- Heavy: > 0.25 and <= 0.50
- Severe: Ratio <= 0.25

AM (7-8)

In [3]:
# # gdf = gpd.read_file(r'../../../inputs/scenario/networks/shapefiles/AM/AM_edges.shp')
# gdf = gpd.read_file(Path(summary_config['sc_run_path'], 'inputs/scenario/networks/shapefiles/AM/AM_edges.shp'))
# gdf.crs = 'EPSG:2285'

# df = process_network_summary()
# # NOTE: for visualization, do not show connectors
# df = df[df['data3'] != 5]
# gdf = gdf.merge(df[df['tod'] == '7to8'][['ij','congestion_index','congestion_category']], left_on='link_id', right_on='ij', how='left')
# gdf['congestion_category'] = gdf['congestion_category'].fillna('Light')
# color_mapping = {'Light': 'grey','Moderate': 'orange','Heavy': 'red','Severe':'black'}
# gdf.plot(color=gdf["congestion_category"].map(color_mapping), alpha=0.3, linewidth=gdf['lanes'], figsize=(20,20)) 

In [4]:
df = process_network_summary()
df = df[df['county']!='Outside Region']

VMT

In [5]:
pd.options.display.float_format = '{:0,.0f}'.format
df_am = df[df['tod'] == '7to8']
_df = df_am.pivot_table(index='congestion_category',columns='county',
               aggfunc='sum',values='VMT')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df['Region'] = _df.sum(axis=1)
_df.loc['Heavy + Severe'] = _df.loc[['Heavy','Severe']].sum(axis=0)
_df

# _df

,King,Kitsap,Pierce,Snohomish,Region
Light,"2,104,827","285,786","1,005,472","887,912","4,283,997"
Moderate,"536,686","7,067","160,135","154,206","858,094"
Heavy,"520,780","9,679","73,711","88,056","692,225"
Severe,"56,661",671,"4,245","5,663","67,239"
Heavy + Severe,"577,440","10,350","77,956","93,718","759,465"


----
Share of VMT

In [6]:

pd.options.display.float_format = '{:0,.1%}'.format
_df = _df/_df.sum(axis=0)
_df

,King,Kitsap,Pierce,Snohomish,Region
Light,55.4%,91.1%,76.1%,72.2%,64.3%
Moderate,14.1%,2.3%,12.1%,12.5%,12.9%
Heavy,13.7%,3.1%,5.6%,7.2%,10.4%
Severe,1.5%,0.2%,0.3%,0.5%,1.0%
Heavy + Severe,15.2%,3.3%,5.9%,7.6%,11.4%


VMT

In [7]:
pd.options.display.float_format = '{:0,.0f}'.format
df_pm = df[df['tod'] == '17to18']
_df = df_pm.pivot_table(index='congestion_category',columns='county',
               aggfunc='sum',values='VMT')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df['Region'] = _df.sum(axis=1)
_df.loc['Heavy + Severe'] = _df.loc[['Heavy','Severe']].sum(axis=0)
_df

,King,Kitsap,Pierce,Snohomish,Region
Light,"2,447,234","322,023","1,149,676","1,002,870","4,921,803"
Moderate,"613,564","6,249","155,523","167,598","942,933"
Heavy,"343,148","9,522","61,616","65,861","480,147"
Severe,"37,583",853,"4,864","5,252","48,552"
Heavy + Severe,"380,732","10,375","66,480","71,112","528,699"


----
Share of VMT

In [8]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df/_df.sum(axis=0)
_df

,King,Kitsap,Pierce,Snohomish,Region
Light,64.0%,92.3%,79.9%,76.4%,71.1%
Moderate,16.1%,1.8%,10.8%,12.8%,13.6%
Heavy,9.0%,2.7%,4.3%,5.0%,6.9%
Severe,1.0%,0.2%,0.3%,0.4%,0.7%
Heavy + Severe,10.0%,3.0%,4.6%,5.4%,7.6%


## Congestion by Road Type

VMT

In [9]:
pd.options.display.float_format = '{:0,.0f}'.format
df_am = df[df['tod'] == '7to8']
_df = df_am.pivot_table(index='congestion_category',columns='facility_type',
               aggfunc='sum',values='VMT')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.loc['Heavy + Severe'] = _df.loc[['Heavy','Severe']].sum(axis=0)
_df

,Arterial,Connector,Highway,Other
Light,"2,115,888","325,123","1,842,686",300
Moderate,"322,264",NaN,"535,791",39
Heavy,"190,377",NaN,"501,848",0
Severe,"32,613",NaN,"34,626",0
Heavy + Severe,"222,991",0,"536,474",0


----
Share of VMT

In [10]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df/_df.sum(axis=0)
_df

,Arterial,Connector,Highway,Other
Light,73.4%,100.0%,53.4%,88.4%
Moderate,11.2%,NaN,15.5%,11.6%
Heavy,6.6%,NaN,14.5%,0.0%
Severe,1.1%,NaN,1.0%,0.0%
Heavy + Severe,7.7%,0.0%,15.5%,0.0%


VMT

In [11]:

pd.options.display.float_format = '{:0,.0f}'.format
df_pm = df[df['tod'] == '17to18']
_df = df_pm.pivot_table(index='congestion_category',columns='facility_type',
               aggfunc='sum',values='VMT')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.loc['Heavy + Severe'] = _df.loc[['Heavy','Severe']].sum(axis=0)
_df

,Arterial,Connector,Highway,Other
Light,"2,435,920","441,743","2,043,736",404
Moderate,"332,625",NaN,"610,308",0
Heavy,"186,303",NaN,"293,838",5
Severe,"32,369",NaN,"16,183",0
Heavy + Severe,"218,672",0,"310,021",5


----
Share of VMT

In [12]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df/_df.sum(axis=0)
_df

,Arterial,Connector,Highway,Other
Light,76.0%,100.0%,62.4%,97.4%
Moderate,10.4%,NaN,18.6%,0.0%
Heavy,5.8%,NaN,9.0%,1.3%
Severe,1.0%,NaN,0.5%,0.0%
Heavy + Severe,6.8%,0.0%,9.5%,1.3%


In [13]:
# _df = df_am.pivot_table(index='congestion_category',columns='facility_type',
#                aggfunc='sum',values='length')
pd.options.display.float_format = '{:0,.0f}'.format
df_pm['length'].sum()

np.float64(15257.412175647827)

## Miles of Roadway withy Heavy Congestion

In [14]:
print(f"Total Miles of Roadway: {int(df_am['length'].sum())}")

Total Miles of Roadway: 15257


In [15]:
congested_len = df_am[df_am['congestion_category'].isin(['Heavy','Severe'])]['length'].sum()
print(f"Total Miles of Roadway: {int(congested_len)}")

Total Miles of Roadway: 460


In [16]:
print(f"Share Congested: {congested_len/df_pm['length'].sum():.1%}")

Share Congested: 3.0%


## Corridor Travel Time

In [17]:
pd.options.display.float_format = '{:0,.1f}'.format

df = pd.read_csv(r'..\..\..\..\outputs\validation\corridor_speeds.csv')

# For each Corridor_Descrption, define as AM or pm beak by finding which direction has max value
df = df[df['tod'].isin(['7to8','17to18']) & (df['Corridor_Number'] <= 20)]

# Corridors are grouped so that directions are [1,2], [3,4]
for i in range(1,21):
# for i in range(7,8):
    _df = df[df['Corridor_Number'] == i]
#     print(_df)
    peak_time = _df.sort_values('auto_time').iloc[-1]['tod']
    df.loc[df['Corridor_Number']==i, 'peak'] = peak_time
    
df.rename(columns={'Corridor_Description': 'Corridor', 'auto_time':'Auto Travel Time (min.)'}, inplace=True)
df[(df['peak'] == '7to8') & (df['tod'] == '7to8')][['Corridor','Auto Travel Time (min.)']]

,Corridor,Auto Travel Time (min.)
15,Everett to Seattle - SB,60.7
24,Tacoma to Seattle - NB,65.8
51,Lynnwood to Bellevue - SB,37.0
60,Tukwila to Bellevue - NB,30.1
78,Auburn to Renton - NB,25.2
96,Redmond to Seattle - WB,28.6
114,Redmond to Bellevue - WB,12.5
132,Issaquah to Bellevue - WB,13.9
159,Bellevue to Seattle via 520 - EB,23.1
168,Bellevue to Seattle via 90 - WB,27.8


In [18]:
df[(df['peak'] == '17to18') & (df['tod'] == '17to18')][['Corridor','Auto Travel Time (min.)']]

,Corridor,Auto Travel Time (min.)
2,Seattle to Everett - NB,52.8
29,Seattle to Tacoma - SB,56.0
38,Bellevue to Lynnwood - NB,31.7
65,Bellevue to Tukwila - SB,27.5
83,Renton to Auburn - SB,21.3
101,Seattle to Redmond - EB,23.0
119,Bellevue to Redmond - EB,10.8
137,Bellevue to Issaquah - EB,12.0
146,Seattle to Bellevue via 520 - WB,17.0
173,Seattle to Bellevue via 90 - EB,24.5


## Delay

In [19]:
df = process_network_summary()
df = df[df['county']!='Outside Region']

In [20]:
df[['total_delay','county']].groupby('county').sum()[['total_delay']]

,total_delay
county,
King,"149,321.8"
Kitsap,"2,726.5"
Pierce,"29,944.7"
Snohomish,"29,737.5"


In [21]:
# Congestion on Transit Routes
df_network = process_network_summary()
# df = df[df['county']!='Outside Region']
# Get PM congestion only for now

tod = '17to18'
df_network_pm = df_network[df_network['tod']==tod]

df = pd.read_csv((Path(output_path, 'transit/transit_segment_results.csv')))
df = df[df['tod']==tod]

df = df[~df['j_node'].isnull()]
df = df.merge(df_network_pm, on=['i_node','j_node'], how='left')

In [22]:
df["delay"] = df["auto_time"] - df["freeflow_time"]
df["person_delay"] = df['segment_volume']*df['delay']

In [23]:
df_sum = df.groupby('line_id').sum()[['auto_time','freeflow_time','person_delay']]
df_sum['vehicle_delay'] = df_sum['auto_time']-df_sum['freeflow_time']

# Merge the line IDs to transit line results to get agency IDs or some other metric
df_line_results = pd.read_csv((Path(output_path, 'transit/transit_line_results.csv')))
df_line_results = df_line_results[df_line_results['tod']==tod]
df_sum = df_sum.merge(df_line_results, on='line_id', how='left')

In [24]:
# Compare person-delay in transit vs person-delay in vehicles on specific corridors

In [25]:
# # Total minutes of delay by agency
# df = df_sum.groupby('agency_code').sum()[['delay']]
# df.index = df.index.astype('int').astype('str').map(summary_config['agency_lookup'])
# df.index.name = 'Agency'
# df.rename(columns={'delay': 'Total Route Delay (min) at PM Peak'}, inplace=True)
# df

In [26]:
# # Ridership and Delay
# df_sum.groupby('agency_code').sum()[['person_delay']]/60

In [27]:
# # df = pd.read_csv((Path(output_path, 'transit/transit_segment_results.csv')))
# # df = df[df['tod']=='7to8']
# df_sum
